In [2]:
import os 
spark_version = 'spark-3.2.1' # most recent version of spark3.0
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BigData_Challenge16").getOrCreate() 

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:15 http://archive.ubuntu.com/ubu

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [12]:
### Create new DataFrame or table to retrieve all the rows where: total_votes >= 20 
df_1 = df.filter(df.total_votes >= 20)
df_1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12257412|R1DOSHH6AI622S|1844161560|     253182049|Eisenhorn (A Warh...|           Books|          4|            1|         22|   N|                N| this isn't a review|never read it-a y...| 2005-10-14|
|         US|   48541186|R1VE0FQQ0QTQJN|0300108834|     915891133|A Little History ...|           Books|          5|    

In [13]:
df_2 = df_1.filter(((df_1.helpful_votes/df_1.total_votes)*100) >= 50)
df_2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48541186|R1VE0FQQ0QTQJN|0300108834|     915891133|A Little History ...|           Books|          5|           16|         20|   N|                Y|Simple, entertain...|Never been much f...| 2005-10-14|
|         US|   13503149|R3MD5TZJGIPTDU|1585424307|     915155182|Millionaire Repub...|           Books|          5|    

In [16]:
#create a new DataFrame or table that retrieves from df_2 where vine == 'Y'
paid_df = df_2.filter(df_2.vine == "Y")
paid_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [17]:
unpaid_df = df_2.filter(df_2.vine == "N")
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48541186|R1VE0FQQ0QTQJN|0300108834|     915891133|A Little History ...|           Books|          5|           16|         20|   N|                Y|Simple, entertain...|Never been much f...| 2005-10-14|
|         US|   13503149|R3MD5TZJGIPTDU|1585424307|     915155182|Millionaire Repub...|           Books|          5|    

In [25]:
##PAID### 

#Total Number of Reviews
paid_review_count = paid_df.count()
print(paid_review_count)

#The Number of 5-Star Reviews
paid_5review = paid_df.filter(paid_df.star_rating == 5).count()
print(paid_5review)

#Percentage of 5-Star Reviews
try:
  paid_5review_percent = (paid_5review/paid_review_count) * 100
except ZeroDivisionError:
    paid_5review_percent = 0
print(paid_5review_percent)



0
0
0


In [27]:
###UNPAID###

#Total Number of Reviews
unpaid_review_count = unpaid_df.count()
print(unpaid_review_count)

#The Number of 5-Star Reviews
unpaid_5review = unpaid_df.filter(unpaid_df.star_rating == 5).count()
print(unpaid_5review)
#Percentage of 5-Star Reviews
try:
  unpaid_5review_percent = (unpaid_5review/unpaid_review_count) * 100
except ZeroDivisionError:
    unpaid_5review_percent = 0

print(unpaid_5review_percent)

403807
242889
60.14977452099642
